In [1]:
import functools

from qdrant_client import QdrantClient

from langchain_groq import ChatGroq
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

import torch
from transformers import AutoModel
from FlagEmbedding import BGEM3FlagModel

from RAGModule.retrieve import retrieve_relevant_chunks

from GraphModule.chains import query_analyzer_chain, rewriter_chain, responder_chain
from GraphModule.pydantic_models import State
from GraphModule.nodes import *
from GraphModule.routes import *

from llm_compiler.utils import *
from llm_compiler.runnables import *
from llm_compiler.prompts import llm_compiler_prompt, joiner_prompt
from llm_compiler.pydantic_models import JoinOutputs

torch.set_grad_enabled(False)

from dotenv import load_dotenv
load_dotenv()

/home/taiquach123471/anaconda3/envs/insomnia_chatbot_v2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
client = QdrantClient("http://localhost:6333")
llm = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)
bge_m3 = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
jina_v3 = AutoModel.from_pretrained("jinaai/jina-embeddings-v3", trust_remote_code=True).to('cuda')

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 119495.84it/s]
/home/taiquach123471/anaconda3/envs/insomnia_chatbot_v2/lib/python3.11/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issu

In [3]:
retrieve_relevant_chunks = functools.partial(retrieve_relevant_chunks, jina_embedding = jina_v3, bge_embedding = bge_m3, client = client)

@tool
def retrieve_chunks(query: str) -> str:
    """retrieve_chunks(query="the search query") - This tool retrieves relevant information from a vector store containing insomnia-related data based on the given query"""
    relevant_chunks = retrieve_relevant_chunks(query=query)
    relevant_chunks = format_chunks(relevant_chunks[:3])
    return relevant_chunks
tools = [retrieve_chunks]

In [4]:
planner = create_planner(llm, tools, llm_compiler_prompt)

In [5]:
runnable = joiner_prompt | llm.with_structured_output(JoinOutputs)
joiner_chain = select_recent_messages | runnable

In [6]:
@as_runnable
def plan_and_schedule(state: State):
    messages = state["llm_compiler_messages"]
    tasks = planner.stream(messages)
    # Begin executing the planner immediately
    try:
        tasks = itertools.chain([next(tasks)], tasks)
    except StopIteration:
        # Handle the case where tasks is empty.
        tasks = iter([])
    scheduled_tasks = schedule_tasks.invoke(
        {
            "messages": messages,
            "tasks": tasks,
        }
    )
    print(scheduled_tasks)
    return {"llm_compiler_messages": scheduled_tasks}

def joiner(state: State):
    join_outputs: JoinOutputs = joiner_chain.invoke({"llm_compiler_messages": state["llm_compiler_messages"]})
    if join_outputs.should_replan:
        return {"llm_compiler_messages":[AIMessage(content = f"Thought: {join_outputs.thought}")] +  [SystemMessage(content = join_outputs.replan_analysis)]}
    else:
        messages = state["llm_compiler_messages"]
        documents = ''
        for msg in messages[::-1]:
            if isinstance(msg, HumanMessage):
                break
            if msg.name == 'join':
                continue
            documents += msg.content + '\n\n'
        
        return {"documents": documents}

In [7]:
query = "What are the most common causes of insomnia in teenagers, and is caffeine consumption one of them?"
tool_messages = plan_and_schedule.invoke({"llm_compiler_messages": [HumanMessage(content=query)]})['llm_compiler_messages']

Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)


[FunctionMessage(content='---Begin Chunk---\nSource: ./Data/extracted/TÁC HẠI/ijerph-18-12389.md\ndoc_id:63250589-0e64-4fc4-b01c-f0217883214e\nTitle: Effects of Caffeine and Caffeinated Beverages in Children, Adolescents and Young Adults: Short Review\n\nContent:\nNumerous studies have pointed out the adverse health effects of EDs and their connection to destructive behaviors. ED consumption is also widespread among American teenagers (13–17 years), and the rate depends on demographic, psychosocial, lifestyle, and substance abuse factors. In a representative American study, 40% of adolescents reported daily consumption. There is a strong correlation between the consumption of EDs and increased soft drug use, which in turn is linked to increasing drug abuse in general. Consumption of EDs among adolescents is also associated with other potentially negative health and behavioral outcomes, such as sensation-seeking, the use of tobacco and other harmful substances, and binge-drinking, as we

In [8]:
join_outputs: JoinOutputs = joiner_chain.invoke({"llm_compiler_messages": [HumanMessage(content=query)] + tool_messages})

In [9]:
chat_history = [HumanMessage(content=query)] + tool_messages + [AIMessage(content = f"Thought: {join_outputs.thought}")] +  [SystemMessage(content = join_outputs.replan_analysis)]
chat_history

[HumanMessage(content='What are the most common causes of insomnia in teenagers, and is caffeine consumption one of them?', additional_kwargs={}, response_metadata={}),
 FunctionMessage(content='---Begin Chunk---\nSource: ./Data/extracted/TÁC HẠI/ijerph-18-12389.md\ndoc_id:63250589-0e64-4fc4-b01c-f0217883214e\nTitle: Effects of Caffeine and Caffeinated Beverages in Children, Adolescents and Young Adults: Short Review\n\nContent:\nNumerous studies have pointed out the adverse health effects of EDs and their connection to destructive behaviors. ED consumption is also widespread among American teenagers (13–17 years), and the rate depends on demographic, psychosocial, lifestyle, and substance abuse factors. In a representative American study, 40% of adolescents reported daily consumption. There is a strong correlation between the consumption of EDs and increased soft drug use, which in turn is linked to increasing drug abuse in general. Consumption of EDs among adolescents is also associa

In [10]:
res = plan_and_schedule.invoke({"llm_compiler_messages": chat_history})

AAAAAAAAAAAAAAAAA


Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
Failed to use model_dump to serialize <class 'langchain_core.tools.structured.StructuredTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)


[FunctionMessage(content='---Begin Chunk---\nSource: ./Data/extracted/TÁC HẠI/JSR-30-e13190.md\ndoc_id:cb9d84ca-cdfb-4684-ba3a-37ed0a595ee9\nTitle: Insomnia symptoms and non-suicidal self-injury in adolescence: understanding temporal relations and mechanisms\n\nContent:\nThe present study examined the temporal links between insomnia symptoms and NSSI during adolescence, showing that these two problems are related over time, in a cyclical fashion. In addition, the present study provided a deeper understanding of the mechanisms that could explain these associations. Specifically, we showed that adolescents who reported insomnia symptoms were more likely to report depressive symptoms 1 year later. This, in turn, made them more likely to engage in NSSI.\n\nWith a few exceptions, most studies examining the link between sleep difficulties and NSSI are cross-sectional in nature. In addition, these studies have used dichotomised measures to assess NSSI or short and very broad measures of sleep